<a href="https://colab.research.google.com/github/Nataliawijayaa/Music_Project_Analysis/blob/main/Music_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Y.Music

# Table of Contents <a id='back'></a>

* [Introduction](#intro)
* [Phase 1. Data Overview](#data_review)
    * [Data Review Conclusion](#data_review_conclusions)
* [Phase 2. Data Preprocessing](#data_preprocessing)
    * [2.1 Header Style](#header_style)
    * [2.2 Missing Values](#missing_values)
    * [2.3 Duplicates](#duplicates)
    * [2.4 Data Preprocessing Conclusion](#data_preprocessing_conclusions)
* [Phase 3. Hypotheses](#hypotheses)
    * [3.1 Hypothesis 1: User activity in Springfield and Shelbyville](#activity)
    * [3.2 Hypothesis 2: Music preference on Monday and Friday](#week)
    * [3.3 Hypothesis 3: Genre preference in Springfield and Shelbyville](#genre)
* [Findings](#end)

## Introduction <a id='intro'></a>
In this project, I will analyze and contrast the music preferences of users residing in the cities of Springfield and Shelbyville. By examining real data from Y.Music, I will test the hypotheses and compare the user behavior between these two cities.

### Objectives: 
To test three hypotheses:
1. User activity varies depending on the day and city.
1. Springfield and Shelbyville residents exhibit distinct genre preferences during Monday mornings and Friday evenings.
1. Musical preferences differ between listeners in Springfield and Shelbyville. Pop music is favored in Springfield, whereas rap music has more fans in Shelbyville.

### Phase
Information regarding user behavior is stored in *file* `/datasets/music_project_en.csv`. As the quality of the data is unknown, it is necessary to verify it before proceeding with hypothesis testing.

Initially, my task is to assess the data quality and determine the extent of any significant issues. Subsequently, during the data pre-processing phase, my focus will be on tackling the most pressing problems that arise.

Therefore, this project will consist of three phases:
 1. Data Overview
 2. Data Preprocessing
 3. Hypotheses Testing

 
[Return to Table of Contents](#back)

## Phase 1. Data Overview <a id='data_review'></a>


In [2]:
import pandas as pd


In [4]:
# read file and store to df
df = pd.read_csv('/content/music_project_en.csv')

In [5]:
# Show the first 10 row
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


In [6]:
# Information about the table
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


The number of values in each column is different. All of which share the data type, which is: `object`.

Based on the ocumentation:
- `'userID'` 
- `'Track'` — track title
- `'artist'` — artist name
- `'genre'`
- `'City'`
- `'time'` — The duration of song playback
- `'Day'` — days in a week

There are three issues with the column name styling:
1. Inconsistency in capitalization, with some names written in uppercase and others in lowercase.
1. Presence of spaces in certain names.
1. Some have two-word column names and lack of proper separation.

Moreover, the varying number of column values indicates the presence of missing data in our dataset.


### Conclusion <a id='data_review_conclusions'></a> 

Every row in the table contains information about a specific song track being played. Some columns hold data describing the track, such as title, artist, and genre. Other columns contain user-related information, such as their hometown and the time when they played the track.


It is evident that the available data is adequate for hypothesis testing. However, we do have missing values.


To proceed with the analysis, it is necessary to perform data pre-processing as a preliminary step.

[Return to Table of Contents](#back)

## Phase 2. Data Preprocessing <a id='data_preprocessing'></a>
To begin, we need to address the formatting inconsistencies in the column headings and resolve any missing values. Afterwards, we should check for duplicates within the dataset to ensure data integrity.

### Column Headings <a id='header_style'></a>


In [7]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Modify the column names according to the rules of good writing style:

* If a name consists of multiple words, use snake_case.
* All characters should be in lowercase.
* Remove spaces

In [8]:
df = df.rename(
    columns = {
        '  userID' : 'user_id',
        'Track' : 'track',
        '  City  ' : 'city',
        'Day' : 'day'
    })

In [9]:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

[Return to Table of Contents](#back)

### Missing Values <a id='missing_values'></a>
First, find the total number of missing values in the table.

In [10]:
df.isna().sum()

user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64

Not all missing values have an impact on the study. For example, missing values in the `track` and `artist` columns are not crucial. We can simply replace them with a clear indicator.

However, missing values in the `genre` column can affect the comparison of music preferences in Springfield and Shelbyville. In real-life scenarios, it is valuable to investigate the reasons behind the missing data and attempt to rectify it. Unfortunately, we don't have that opportunity in this project. Therefore, I will:
* Fill in the missing values with an indicator.
* Evaluate the extent to which missing values can impact the calculations.

In [11]:
# loop the column name and substitute the missing values with 'unknown'    
columns_to_replace = ['track', 'artist', 'genre']

for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

In [12]:
# check the missing values
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

[Return to Table of Contents](#back)

### Duplicates <a id='duplicates'></a>


In [13]:
# Calculate explicit duplicates
df.duplicated().sum()

3826

In [14]:
# Delete explicit duplicates
df = df.drop_duplicates()

In [15]:
# Check the duplicates
df.duplicated().sum()

0

Now, remove implicit duplicates in the `genre` column. For example, genre names may be written in different ways. Such errors can also affect the results.

Display *list* that contains unique genre names, then sort the list alphabetically. To do this:
* Retrieve the specified DataFrame column.
* Apply a sorting method to the column.
* For the sorted column, call a method that will yield all unique values in the column.

In [16]:
# View unique genre names.
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'children', 'chill', 'chinese', 'choral', 'christian', 'christmas',
       'classical', 'classicmetal', 'club', 'colombian', 'comedy',
       'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic', 'eurofo

Take a close look at the *list* to identify implicit duplicates of the genre `hiphop`. These duplicates may include inaccurately written names or alternative names for the same genre.

We will observe the following implicit duplicates:
* `hip`
* `hop`
* `hip-hop`

In [17]:
# Function to replace implicit duplicates
def replace_wrong_genres(wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre)

In [18]:
# Insert a function that replaces implicit duplicates
duplicates = ['hip', 'hop', 'hip-hop']
genre = 'hiphop'
replace_wrong_genres(duplicates, genre)

In [19]:
# Check for implicit duplicates
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'children', 'chill', 'chinese', 'choral', 'christian', 'christmas',
       'classical', 'classicmetal', 'club', 'colombian', 'comedy',
       'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic', 'eurofo

[Return to Table of Contents](#back)

### Conclusion <a id='data_preprocessing_conclusions'></a>
We have identified three issues in our data:
- Incorrect title formatting
- Missing values
- Explicit and implicit duplicates

The column titles have been cleaned for easier table processing.
All missing values have been replaced with `'unknown'`. However, we still need to assess if the missing values in the `'genre'` column will impact our calculations.

The absence of duplicates will provide more accurate and easily interpretable results.

Now, we can proceed to hypothesis testing.

[Return to Table of Contents](#back)

## Phase 3. Hypotheses Testing <a id='hypotheses'></a>

### Hypothesis 1: User activity in Springfield and Shelbyville <a id='activity'></a>

According to the first hypothesis, users from Springfield and Shelbyville have behavioral differences in listening to music. This testing uses data collected from three days of the week: Monday, Wednesday, and Friday.

* Divide the users into several groups based on the city.
* Compare the number of tracks played by each group on Monday, Wednesday, and Friday.

Evaluate user activity in each city. Group the data based on the city and find the number of tracks played in each group.



In [20]:
# Calculate the number of tracks played in each city.
df.groupby('city')['track'].count()

city
Shelbyville    18512
Springfield    42741
Name: track, dtype: int64

Users from Springfield play more tracks than users from Shelbyville. However, this does not indicate that the residents of Springfield listen to music more frequently. Springfield is a larger city with a larger user base.

Now, group the data based on the day and find the number of tracks played on Monday, Wednesday, and Friday.


In [21]:
# Calculate the number of tracks played on each day.
df.groupby('day')['track'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

Wednesday is the overall calmest day. However, if we consider the two cities separately, we might come to different conclusions.

We have observed how grouping works based on city or day. Now, let's write a function that will group the data based on city and day.

Create a function called `number_tracks()` to calculate the number of tracks played for a specific day and city. The function will require two parameters:
* The name of the day in a week.
* The name of the city.

In the function we are creating, use a variable to store the rows from the original table, where:
* The value in the 'day' column is equal to the day parameter.
* The value in the 'city' column is equal to the city parameter.

Apply sequential filtering using logical indexing.

Then, calculate the number of `'user_id'` values in the resulting table. Store the result in a new variable. Finally, return this variable from the function.

In [22]:
# <Creating the function number_tracks()>
# We will declare a function with two parameters: day=, city=.
# Make the variable track_list store the rows of the df, where
# the value in the 'day' column is equal to the day= parameter, and at the same time,
# the value in the 'city' column is equal to the city= parameter (apply sequential filtering with logical indexing).
# Make the variable track_list_count store the count of the 'user_id' column values in track_list (using the count() method).
# Ensure that the function produces a number: the value of track_list_count.

def number_tracks(day, city):
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count


# The function calculates the number of tracks played for a specific city and day.
# First, it retrieves the rows with the desired day from the table,
# then it filters those rows with the desired city,
# then it calculates the count of 'user_id' values in the filtered table,
# and finally, it returns that count.
# To see the result, wrap the function call in a print() statement.


Call `number_tracks()` six times and change the parameter values in each call so we can retrieve data for both cities for each day (Monday, Wednesday, and Friday).

In [23]:
# The number of tracks played in Springfield on Monday
print(number_tracks('Monday', 'Springfield'))

15740


In [24]:
# The number of tracks played in Shelbyville on Monday
print(number_tracks('Monday', 'Shelbyville'))

5614


In [25]:
#  The number of tracks played in Springfield on Wednesday
print(number_tracks('Wednesday', 'Springfield'))

11056


In [26]:
#  The number of tracks played in Shelbyville on Wednesday
print(number_tracks('Wednesday', 'Shelbyville'))

7003


In [27]:
# The number of tracks played in Springfield on Friday
print(number_tracks('Friday', 'Springfield'))

15945


In [28]:
# The number of tracks played in Shelbyville on Friday
print(number_tracks('Friday', 'Shelbyville'))

5895


Use `pd.DataFrame` to create a table with the following specifications:
* The column names are: `['city', 'monday', 'wednesday', 'friday']`
* The data is the result of `number_tracks()`


In [29]:
# result table
result = df.groupby(['city', 'day']).size().reset_index()\
.pivot(index='city', columns='day', values=0).reset_index()\
.rename(columns={i:i.lower() for i in ['Friday', 'Monday', 'Wednesday']})
[['city', 'monday', 'wednesday', 'friday']]
print(result)

day         city  friday  monday  wednesday
0    Shelbyville    5895    5614       7003
1    Springfield   15945   15740      11056


**Conclusion**

The data obtained reveals differences in user behavior:
* In Springfield, the number of tracks played reaches its peak on Monday and Friday, while there is a decrease in activity on Wednesday.
* In Shelbyville, on the other hand, users listen music more on Wednesday and user activity is lower on Monday and Friday.

[Return to Table of Contents](#back)

### Hypothesis 2: Music at the Beginning and End of the Week. <a id='week'></a>


According to the second hypothesis, on Monday mornings and Friday evenings, residents of Springfield listen to different music genres compared to the ones enjoyed by residents of Shelbyville.

Get the tables:
* For Springfield — `spr_general`
* For Shelbyville — `shel_general`


In [30]:
# Get the table spr_general from df, where the value in the 'city' column is 'Springfield'.
spr_general = df[df['city'] == 'Springfield']

In [31]:
# Get the table shel_general from df, where the value in the 'city' column is 'Shelbyville'.
shel_general = df[df['city'] == 'Shelbyville']

Write a function called `genre_weekday()` with four parameters:
* A table for the data
* The name of the weekday
* The start time stamp in the format 'hh:mm'
* The end time stamp in the format 'hh:mm'

The function should provide information about the top 15 most popular genres on a specific weekday during the period between the two time stamps.

In [32]:
# Declare the function genre_weekday() with parameters day=, time1=, and time2=. The function should
# provide information about the most popular genres on a specific day and time:
# 1) Make the variable genre_df store the rows that satisfy the following conditions:
#   - the value in the 'day' column is equal to the day= argument value
#   - the value in the 'time' column is greater than the time1= argument value
#   - the value in the 'time' column is less than the time2= argument value
#   Use sequential filtering with logical indexing.
#
# 2) Group genre_df by the 'genre' column, then take one of its columns,
#    and use the count() method to find the count of entries for each
#    represented genre; store the resulting Series in the variable genre_df_count.
#
# 3) Sort genre_df_count in descending order based on frequency and store the result
#    in the variable genre_df_sorted.
#
# 4) Generate a Series object with the first 15 values from genre_df_sorted - the top
#    15 most popular genres (on a specific day, during a specific time period).

def genre_weekday(table, day, time1, time2):

    # sequential filtering
    genre_df = table
    
    # genre_df will only store the rows of df that have the same day value as day.
    genre_df = genre_df[genre_df['day'] == day]

    # genre_df will only store the rows of df that have a time value smaller than time2.
    genre_df = genre_df[genre_df['time'] < time2]

    # genre_df will only store the rows of df that have a time value larger than time1.
    genre_df = genre_df[genre_df['time'] > time1]

    # Group the filtered DataFrame by the column named 'genre', select the 'genre' column, and find the count of rows for each genre using the count() method.
    genre_df_count = genre_df.groupby('genre')['genre'].count()

    # Sort the results in descending order (the most popular genres appear earlier in the Series object).
    genre_df_sorted = genre_df_count.sort_values(ascending = False)

    # Generate a Series object that stores the top 15 most popular genres on a specific day during a specific time period.
    return genre_df_sorted[:15]

Compare the results of the `genre_weekday()` function for Springfield and Shelbyville on Monday morning (from 07:00 to 11:00) and on Friday evening (from 17:00 to 23:00):

In [33]:
# Call the function for Monday morning in Springfield (spr_general)
genre_weekday(spr_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
classical      157
metal          120
jazz           100
folk            97
soundtrack      95
Name: genre, dtype: int64

In [34]:
# Call the function for Monday morning in Shelbyville (shel_general)
genre_weekday(shel_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
world           36
rap             32
soundtrack      31
rnb             27
metal           27
Name: genre, dtype: int64

In [35]:
# Call the function for Friday evening in Springfield
genre_weekday(spr_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
classical      163
alternative    163
rusrap         142
jazz           111
unknown        110
soundtrack     105
rnb             90
metal           88
Name: genre, dtype: int64

In [36]:
# Call the function for Friday evening in Shelbyville
genre_weekday(shel_general, 'Friday', '17:00', '23:00')

genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
unknown         47
ruspop          47
soundtrack      40
metal           39
rap             36
Name: genre, dtype: int64

**Conclusion**

After comparing the top 15 genres on Monday morning, we can draw the following conclusions:

1. Users from Springfield and Shelbyville listen to music with the same genres. The top five genres in both cities are the same, with only rock and electronic genres swapping places.

1. In Springfield, the amount of missing values is surprisingly high, resulting in the `'unknown'` value being ranked 10th. This indicates that the missing values comprise a significant proportion of the data, raising concerns about the reliability of our conclusions.

For Friday evening, the situation is similar. Individual genres vary, but overall, the top 15 genres are the same for both cities.

Therefore, the second hypothesis is partially true:
* Users listen to the same music at the beginning and end of the week.
* There are no noticeable differences between Springfield and Shelbyville. In both cities, pop is the most popular genre.

However, the significance of the missing values raises doubts about these results. In Springfield, there are so many missing values that they affect our top 15 genre results. Without these missing values, the results could be different.

[Return to Table of Contents](#back)

### Hypothesis 3: Genre Preferences in Springfield and Shelbyville. <a id='genre'></a>



Hypothesis: Shelbyville enjoys rap music, while residents of Springfield have a preference for pop.

Group the `spr_general` table by genre and find the count of tracks played for each genre using the `count()` method. Then, sort the results in descending order and save it to `spr_genres`.

In [37]:
# Group the spr_general table based on the 'genre' column, 
# calculate the count of the 'genre' column using count() within the grouping, 
# sort the resulting Series in descending order, and save the result to spr_genres
spr_genres = spr_general.groupby('genre')['track'].count()
spr_genres = spr_genres.sort_values(ascending = False)

In [38]:
# Displaying the first 10 rows of spr_genres
spr_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: track, dtype: int64

Now, let's do the same for the data from Shelbyville.

Group the `shel_general` table by genre and find the count of tracks played for each genre. Then, sort the resulting series in descending order and save the results to the `shel_genres` table.

In [39]:
# Group the shel_general table based on the 'genre' column, 
# calculate the count of the 'genre' column using count() within the grouping, 
# sort the resulting Series in descending order, and save the result to shel_general
shel_genres = shel_general.groupby('genre')['track'].count()
shel_genres = shel_genres.sort_values(ascending = False)

In [40]:
# Displaying the first 10 rows of shel_genres
shel_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: track, dtype: int64

**Conclusion**


This hypothesis is partially confirmed:

* Pop music is the most popular genre in Springfield, as we suspected.
* However, pop music is equally popular in both Springfield and Shelbyville, and rap music does not appear in the top 5 genres for either city.

[Return to the Table of Contents](#back)

# Findings <a id='end'></a>

We have tested the following three hypotheses:
1. User activity in Springfield and Shelbyville is dependent on the day of the week, although these two cities vary in various ways.
1. On Monday mornings, residents of Springfield and Shelbyville listen to different genres. This also applies on Friday nights.
1. Listeners in Springfield and Shelbyville have different preferences. In both Springfield and Shelbyville, they prefer pop music.


After analyzing the available data, we can conclude that:
1. User activity in Springfield and Shelbyville is dependent on the day of the week, despite the differences between the cities.

The first hypothesis is fully accepted.

2. Music preferences do not significantly vary throughout the week in Springfield and Shelbyville. We can observe minor differences in rankings on Monday, but:
* Both in Springfield and Shelbyville, users mostly listen to pop music.

Therefore, this hypothesis cannot be accepted. It is also important to note that the results may differ if we didn't have missing values.

3. It turns out that the music preferences of users from Springfield and Shelbyville are very similar.

The third hypothesis is rejected. If there are indeed differences in preferences, unfortunately, we cannot determine them from this data.

[Return to Table of Contents](#back)